# 🤖 Agente GROZY - Sistema Inteligente de Optimización de Compras

## Agente con IA que integra:
- **Herramientas de Consulta**: Búsqueda semántica de productos
- **Herramientas de Escritura**: Generación de carros de compra personalizados
- **Herramientas de Razonamiento**: Validación nutricional y optimización de presupuesto
- **Memoria**: Contexto conversacional y preferencias del usuario
- **Planificación Adaptativa**: Ajusta estrategias según restricciones y disponibilidad

In [ ]:
# 📦 Instalación de dependencias
# !pip install langchain langchain-openai langchain-community faiss-cpu python-dotenv

In [6]:
# Importaciones y configuración inicial
import json
import os
from datetime import datetime
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional

# LangChain Core
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.runnables import RunnablePassthrough

# LangChain para Agentes (API de LangChain 1.0+)
from langchain.agents import create_agent

# Cargar variables de entorno
load_dotenv()

print("✅ Bibliotecas importadas correctamente")
print(f"📅 Sesión iniciada: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Bibliotecas importadas correctamente
📅 Sesión iniciada: 2025-11-26 16:34:36


## 1️⃣ Carga de Datos y Configuración del Vector Store

In [7]:
# Configuración de credenciales
github_token = os.getenv('GITHUB_TOKEN')
openai_base_url = os.getenv('OPENAI_BASE_URL')

if not github_token:
    print("❌ Error: GITHUB_TOKEN no está configurado")
    raise ValueError("Configura GITHUB_TOKEN en el archivo .env")
else:
    print("✅ Credenciales configuradas")

# Cargar productos
file_path = os.path.join("data", "productos_unimarc_muestra.json")
with open(file_path, "r", encoding="utf-8") as f:
    productos = json.load(f)

print(f"✅ Cargados {len(productos)} productos")

# Convertir a documentos con metadata enriquecida
docs = []
for p in productos:
    contenido = (
        f"Nombre: {p['nombre']} | "
        f"Precio: ${p['precio']} | "
        f"Categoría: {p['categoria']} | "
        f"Subcategoría: {p.get('subcategoria', 'N/A')} | "
        f"Supermercado: {p['supermercado']}"
    )
    docs.append(Document(
        page_content=contenido,
        metadata={
            "nombre": p['nombre'],
            "precio": p['precio'],
            "categoria": p['categoria'],
            "subcategoria": p.get('subcategoria', 'N/A'),
            "supermercado": p['supermercado']
        }
    ))

# Crear embeddings y vector store
embeddings = OpenAIEmbeddings(
    api_key=github_token,
    base_url=openai_base_url,
    model="text-embedding-3-small"
)

print("🔄 Creando vector store (puede tardar)...")
vectorstore = FAISS.from_documents(docs, embeddings)
print("✅ Vector store creado exitosamente")

# Configurar LLM
llm = ChatOpenAI(
    api_key=github_token,
    base_url=openai_base_url,
    model="gpt-4o-mini",
    temperature=0.7
)
print("✅ LLM configurado (gpt-4o-mini)")

✅ Credenciales configuradas
✅ Cargados 495 productos
🔄 Creando vector store (puede tardar)...
✅ Vector store creado exitosamente
✅ LLM configurado (gpt-4o-mini)


## 2️⃣ Definición de Herramientas del Agente

### A. Herramientas de Consulta 🔍

In [11]:
@tool
def buscar_productos(query: str, k: int = 10) -> str:
    """Busca productos en la base de datos usando búsqueda semántica.
    
    Args:
        query: Descripción de los productos a buscar (ej: 'frutas frescas', 'lácteos bajos en grasa')
        k: Número máximo de productos a retornar (default: 10)
    
    Returns:
        Lista de productos encontrados con nombre, precio y categoría
    """
    results = vectorstore.similarity_search(query, k=k)
    productos_encontrados = []
    
    for doc in results:
        productos_encontrados.append(
            f"- {doc.metadata['nombre']} | ${doc.metadata['precio']} | {doc.metadata['categoria']}"
        )
    
    return f"Productos encontrados ({len(productos_encontrados)}):\n" + "\n".join(productos_encontrados)


@tool
def obtener_estadisticas_categorias() -> str:
    """Obtiene estadísticas de productos por categoría y precios promedio.
    
    Returns:
        Resumen estadístico de categorías disponibles
    """
    categorias = {}
    precios_por_cat = {}
    
    for p in productos:
        cat = p['categoria']
        categorias[cat] = categorias.get(cat, 0) + 1
        if cat not in precios_por_cat:
            precios_por_cat[cat] = []
        precios_por_cat[cat].append(p['precio'])
    
    resultado = "📊 ESTADÍSTICAS DE PRODUCTOS:\n\n"
    resultado += f"Total productos: {len(productos)}\n"
    resultado += f"Categorías: {len(categorias)}\n\n"
    resultado += "Por categoría:\n"
    
    for cat, count in sorted(categorias.items(), key=lambda x: x[1], reverse=True)[:8]:
        precio_prom = sum(precios_por_cat[cat]) / len(precios_por_cat[cat])
        resultado += f"  • {cat}: {count} productos (promedio ${precio_prom:,.0f})\n"
    
    return resultado

print("✅ Herramientas de consulta definidas")

✅ Herramientas de consulta definidas


### B. Herramientas de Razonamiento 🧠

In [12]:
@tool
def validar_dieta(productos_seleccionados: str, tipo_dieta: str) -> str:
    """Valida si los productos seleccionados cumplen con restricciones dietéticas.
    
    Args:
        productos_seleccionados: Lista de nombres de productos separados por coma
        tipo_dieta: Tipo de dieta ('vegetariana', 'diabetica', 'fitness', 'celiaca')
    
    Returns:
        Análisis de compatibilidad con la dieta
    """
    productos_lista = [p.strip() for p in productos_seleccionados.split(',')]
    
    restricciones = {
        'vegetariana': ['carne', 'pollo', 'pavo', 'pescado', 'atun', 'salmon', 'cerdo'],
        'diabetica': ['azucar', 'dulce', 'chocolate', 'caramelo', 'bebida', 'jugo'],
        'fitness': [],  # Sin restricciones específicas
        'celiaca': ['pan', 'trigo', 'harina', 'pasta', 'galleta']
    }
    
    palabras_prohibidas = restricciones.get(tipo_dieta.lower(), [])
    problemas = []
    
    for producto in productos_lista:
        for palabra in palabras_prohibidas:
            if palabra in producto.lower():
                problemas.append(f"⚠️ '{producto}' contiene '{palabra}' (no compatible con dieta {tipo_dieta})")
    
    if not problemas:
        return f"✅ Todos los productos son compatibles con la dieta {tipo_dieta}"
    else:
        return "Problemas encontrados:\n" + "\n".join(problemas)


@tool
def calcular_presupuesto(productos_seleccionados: str, presupuesto_max: float) -> str:
    """Calcula el costo total y verifica si está dentro del presupuesto.
    
    Args:
        productos_seleccionados: Lista de nombres de productos separados por coma
        presupuesto_max: Presupuesto máximo disponible en CLP
    
    Returns:
        Análisis de costos y cumplimiento de presupuesto
    """
    productos_lista = [p.strip().lower() for p in productos_seleccionados.split(',')]
    total = 0
    productos_con_precio = []
    
    for nombre_prod in productos_lista:
        # Buscar producto en base de datos
        encontrado = False
        for p in productos:
            if nombre_prod in p['nombre'].lower():
                total += p['precio']
                productos_con_precio.append(f"  • {p['nombre']}: ${p['precio']:,}")
                encontrado = True
                break
        
        if not encontrado:
            productos_con_precio.append(f"  • {nombre_prod}: ⚠️ No encontrado")
    
    resultado = "💰 ANÁLISIS DE PRESUPUESTO:\n\n"
    resultado += "\n".join(productos_con_precio)
    resultado += f"\n\nTotal: ${total:,} CLP\n"
    resultado += f"Presupuesto máximo: ${presupuesto_max:,} CLP\n"
    
    if total <= presupuesto_max:
        resultado += f"✅ Dentro del presupuesto (${presupuesto_max - total:,} disponibles)"
    else:
        resultado += f"❌ Sobrepasa el presupuesto por ${total - presupuesto_max:,}"
    
    return resultado


@tool
def evaluar_balance_nutricional(productos_seleccionados: str) -> str:
    """Evalúa el balance nutricional de los productos seleccionados.
    
    Args:
        productos_seleccionados: Lista de nombres de productos separados por coma
    
    Returns:
        Análisis del balance entre categorías nutricionales
    """
    productos_lista = [p.strip().lower() for p in productos_seleccionados.split(',')]
    
    categorias_nutricionales = {
        'Proteínas': 0,
        'Frutas/Verduras': 0,
        'Lácteos': 0,
        'Cereales': 0,
        'Otros': 0
    }
    
    for nombre_prod in productos_lista:
        for p in productos:
            if nombre_prod in p['nombre'].lower():
                cat = p['categoria'].lower()
                if 'carne' in cat or 'pescado' in cat:
                    categorias_nutricionales['Proteínas'] += 1
                elif 'fruta' in cat or 'verdura' in cat:
                    categorias_nutricionales['Frutas/Verduras'] += 1
                elif 'lacteo' in cat or 'queso' in cat:
                    categorias_nutricionales['Lácteos'] += 1
                elif 'pan' in cat or 'cereal' in cat or 'desayuno' in cat:
                    categorias_nutricionales['Cereales'] += 1
                else:
                    categorias_nutricionales['Otros'] += 1
                break
    
    resultado = "🥗 BALANCE NUTRICIONAL:\n\n"
    for categoria, cantidad in categorias_nutricionales.items():
        if cantidad > 0:
            resultado += f"  • {categoria}: {cantidad} productos\n"
    
    # Recomendaciones
    resultado += "\nRecomendaciones:\n"
    if categorias_nutricionales['Frutas/Verduras'] < 3:
        resultado += "  ⚠️ Considera agregar más frutas y verduras\n"
    if categorias_nutricionales['Proteínas'] == 0:
        resultado += "  ⚠️ Falta fuente de proteína\n"
    if categorias_nutricionales['Lácteos'] == 0:
        resultado += "  ⚠️ Considera agregar lácteos\n"
    
    if not any('⚠️' in resultado for _ in range(1)):
        resultado += "  ✅ Balance nutricional adecuado"
    
    return resultado

print("✅ Herramientas de razonamiento definidas")

✅ Herramientas de razonamiento definidas


### C. Herramientas de Escritura ✍️

In [13]:
@tool
def generar_carro_optimizado(tipo_dieta: str, presupuesto: float, personas: int) -> str:
    """Genera un carro de compras optimizado según parámetros.
    
    Args:
        tipo_dieta: Tipo de dieta ('vegetariana', 'diabetica', 'fitness', 'familiar')
        presupuesto: Presupuesto máximo en CLP
        personas: Número de personas
    
    Returns:
        Carro de compras generado con productos, precios y justificación
    """
    # Definir queries según tipo de dieta
    queries_dieta = {
        'vegetariana': 'frutas verduras lácteos huevos legumbres cereales',
        'diabetica': 'verduras carnes magras lácteos sin azúcar cereales integrales',
        'fitness': 'proteínas pollo pavo atún huevos avena frutas verduras',
        'familiar': 'frutas verduras carnes lácteos pan cereales snacks'
    }
    
    query = queries_dieta.get(tipo_dieta.lower(), queries_dieta['familiar'])
    
    # Buscar productos
    num_productos = min(20, int(presupuesto / 1000))  # Estimación
    results = vectorstore.similarity_search(query, k=num_productos * 2)
    
    # Seleccionar productos dentro del presupuesto
    productos_seleccionados = []
    total = 0
    
    for doc in results:
        precio = doc.metadata['precio']
        if total + precio <= presupuesto:
            productos_seleccionados.append({
                'nombre': doc.metadata['nombre'],
                'precio': precio,
                'categoria': doc.metadata['categoria']
            })
            total += precio
            
            if len(productos_seleccionados) >= num_productos:
                break
    
    # Formatear resultado
    resultado = f"🛒 CARRO DE COMPRAS - Dieta {tipo_dieta.upper()}\n"
    resultado += f"👥 Para {personas} personas | 💰 Presupuesto: ${presupuesto:,}\n\n"
    resultado += "Productos seleccionados:\n\n"
    
    for i, p in enumerate(productos_seleccionados, 1):
        resultado += f"{i}. {p['nombre']}\n"
        resultado += f"   💵 ${p['precio']:,} | 📁 {p['categoria']}\n"
    
    resultado += f"\n💰 TOTAL: ${total:,} CLP\n"
    resultado += f"💵 Saldo restante: ${presupuesto - total:,} CLP\n"
    
    return resultado


@tool
def guardar_preferencias_usuario(nombre_usuario: str, preferencias: str) -> str:
    """Guarda las preferencias del usuario para futuras sesiones (memoria largo plazo).
    
    Args:
        nombre_usuario: Identificador del usuario
        preferencias: Preferencias en formato texto (ej: 'vegetariano, presupuesto 30000, 2 personas')
    
    Returns:
        Confirmación de guardado
    """
    # En producción esto iría a una base de datos
    archivo_preferencias = "data/preferencias_usuarios.json"
    
    try:
        if os.path.exists(archivo_preferencias):
            with open(archivo_preferencias, 'r', encoding='utf-8') as f:
                datos = json.load(f)
        else:
            datos = {}
        
        datos[nombre_usuario] = {
            'preferencias': preferencias,
            'fecha': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        with open(archivo_preferencias, 'w', encoding='utf-8') as f:
            json.dump(datos, f, ensure_ascii=False, indent=2)
        
        return f"✅ Preferencias guardadas para '{nombre_usuario}'"
    except Exception as e:
        return f"❌ Error al guardar preferencias: {str(e)}"

print("✅ Herramientas de escritura definidas")

✅ Herramientas de escritura definidas


## 3️⃣ Configuración de Memoria

### Memoria de Corto Plazo (Conversacional)

In [14]:
# Memoria conversacional - En LangGraph la memoria es manejada internamente
# Vamos a usar un sistema de mensajes para mantener el historial

chat_history = []

def add_to_history(role: str, content: str):
    """Agrega un mensaje al historial de conversación."""
    if role == "user":
        chat_history.append(HumanMessage(content=content))
    elif role == "assistant":
        chat_history.append(AIMessage(content=content))

print("✅ Sistema de memoria configurado")

✅ Sistema de memoria configurado


## 4️⃣ Creación del Agente con Planificación Adaptativa

In [15]:
# Lista de todas las herramientas
tools = [
    buscar_productos,
    obtener_estadisticas_categorias,
    validar_dieta,
    calcular_presupuesto,
    evaluar_balance_nutricional,
    generar_carro_optimizado,
    guardar_preferencias_usuario
]

# Prompt del sistema con planificación adaptativa
system_message = """Eres GROZY, un agente inteligente especializado en optimización de compras de supermercado.

TU OBJETIVO: Ayudar a usuarios a crear carros de compra optimizados según sus necesidades.

CAPACIDADES:
1. 🔍 Consulta: Puedes buscar productos y obtener estadísticas
2. 🧠 Razonamiento: Puedes validar dietas, calcular presupuestos y evaluar balance nutricional
3. ✍️ Escritura: Puedes generar carros optimizados y guardar preferencias

PLANIFICACIÓN ADAPTATIVA:
- Si el usuario pide un carro de compras, PRIMERO pregunta: tipo de dieta, presupuesto y número de personas
- Si falta información, solicítala antes de generar el carro
- Si el presupuesto es insuficiente, sugiere alternativas más económicas
- Si hay restricciones dietéticas, valídalas ANTES de finalizar
- Si el balance nutricional es pobre, recomienda productos adicionales

PROCESO RECOMENDADO:
1. Entender necesidades del usuario
2. Verificar disponibilidad de productos (usar buscar_productos)
3. Generar carro inicial (usar generar_carro_optimizado)
4. Validar restricciones (usar validar_dieta)
5. Verificar presupuesto (usar calcular_presupuesto)
6. Evaluar balance (usar evaluar_balance_nutricional)
7. Ajustar si es necesario
8. Presentar resultado final

MEMORIA:
- Recuerda el contexto de la conversación
- Si el usuario menciona preferencias recurrentes, ofrece guardarlas

TONO: Amigable, profesional y proactivo. Usa emojis moderadamente.
"""

# Crear agente usando la nueva API de LangChain 1.0+
agent_executor = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_message,
    debug=False  # Deshabilitado para salida más limpia
)

print("✅ Agente GROZY creado exitosamente")
print(f"📋 Herramientas disponibles: {len(tools)}")

✅ Agente GROZY creado exitosamente
📋 Herramientas disponibles: 7


## 5️⃣ Ejemplos de Uso del Agente

### Ejemplo 1: Consulta Simple

In [16]:
# Invocar el agente con un mensaje
messages = [HumanMessage(content="Hola, ¿qué productos tienes disponibles?")]
respuesta = agent_executor.invoke({"messages": messages})

print("\n" + "="*60)
print("RESPUESTA DEL AGENTE:")
print("="*60)
# La respuesta está en el último mensaje
print(respuesta['messages'][-1].content)


RESPUESTA DEL AGENTE:
¡Hola! Puedo ayudarte a buscar productos específicos. ¿Tienes en mente algún tipo de producto en particular, como frutas, verduras, lácteos, etc.? 🤔


### Ejemplo 2: Generación de Carro con Planificación Adaptativa

In [28]:
messages = [HumanMessage(content="Necesito un carro de compras vegetariano para 3 personas con presupuesto de $25,000")]
respuesta = agent_executor.invoke({"messages": messages})

print("\n" + "="*60)
print("RESPUESTA DEL AGENTE:")
print("="*60)
print(respuesta['messages'][-1].content)


RESPUESTA DEL AGENTE:
Aquí tienes tu carro de compras vegetariano para 3 personas, diseñado con un presupuesto de $25,000:

🛒 **CARRO DE COMPRAS - Dieta VEGETARIANA**
- 👥 **Para 3 personas**
- 💰 **Presupuesto: $25,000**
  
### Productos seleccionados:
1. **Compota vivo veggie pera manzana y espinaca** 120 g - 💵 $800
2. **Yoghurt batido colun light sin lactosa frutilla** 120 g - 💵 $360
3. **Yoghurt sin lactosa loncoleche trozos durazno** 140 g - 💵 $580
4. **Yoghurt sin lactosa surlat colágeno frutilla** 125 g - 💵 $460
5. **Pack bebida láctea chamyto frutilla** 6 un de 80 ml - 💵 $2,090
6. **Pack jalea loncoleche protein guinda** 4 un de 100 g - 💵 $3,120
7. **Yoghurt soprole proteína frutilla** 155 g - 💵 $690
8. **Sémola con leche dila** pote 250 g - 💵 $1,390
9. **Pack yoghurt+cereal mix nestlé chocapic+zucosos** 5 un de 142 g - 💵 $3,440
10. **Jalea uno soprole sabor frambuesa** 4 un de 110 g - 💵 $1,490
11. **Pack yoghurt loncoleche sin lactosa sabores** 8 un de 120 g - 💵 $2,770
12. **Ce

In [17]:
# Ejemplo más directo: Generar carro inmediatamente
messages = [HumanMessage(content="Genera un carro vegetariano para 3 personas con $25000. Usa la herramienta generar_carro_optimizado.")]
respuesta = agent_executor.invoke({"messages": messages})

print("\n" + "="*60)
print("RESPUESTA DEL AGENTE:")
print("="*60)
print(respuesta['messages'][-1].content)


RESPUESTA DEL AGENTE:
Aquí tienes tu carro de compras optimizado para una dieta vegetariana, diseñado para 3 personas y con un presupuesto de $25,000 CLP. 🛒✨

### CARRO DE COMPRAS - Dieta VEGETARIANA
👥 **Para 3 personas** | 💰 **Presupuesto: $25,000.0**

#### Productos seleccionados:

1. **Compota vivo veggie pera manzana y espinaca 120 g**  
   💵 $800 | 📁 Lácteos, Huevos y Refrigerados
2. **Yoghurt batido colun light sin lactosa frutilla 120 g**  
   💵 $360 | 📁 Lácteos, Huevos y Refrigerados
3. **Yoghurt sin lactosa loncoleche trozos durazno 140 g**  
   💵 $580 | 📁 Lácteos, Huevos y Refrigerados
4. **Yoghurt sin lactosa surlat colágeno frutilla 125 g**  
   💵 $460 | 📁 Lácteos, Huevos y Refrigerados
5. **Pack bebida láctea chamyto frutilla 6 un de 80 ml**  
   💵 $2,090 | 📁 Lácteos, Huevos y Refrigerados
6. **Pack jalea loncoleche protein guinda 4 un de 100 g**  
   💵 $3,120 | 📁 Lácteos, Huevos y Refrigerados
7. **Yoghurt soprole proteína frutilla 155 g**  
   💵 $690 | 📁 Lácteos, Huevos

### Ejemplo 3: Validación y Ajuste Adaptativo

### Ejemplo 4: Memoria - Conversación Continua

In [18]:
# Primera interacción
messages1 = [HumanMessage(content="Me llamo Franco y soy vegetariano")]
respuesta1 = agent_executor.invoke({"messages": messages1})
print("Respuesta 1:", respuesta1['messages'][-1].content)

print("\n" + "-"*60 + "\n")

# Segunda interacción (el agente debe recordar - agregamos el historial)
messages2 = messages1 + respuesta1['messages'] + [HumanMessage(content="Arma un carro para mí con $20,000")]
respuesta2 = agent_executor.invoke({"messages": messages2})
print("Respuesta 2:", respuesta2['messages'][-1].content)

Respuesta 1: ¡Hola Franco! 😊 Es genial saber que eres vegetariano. Para ayudarte a crear un carro de compras optimizado, necesitaré un poco más de información:

1. ¿Cuál es tu presupuesto?
2. ¿Para cuántas personas es el carro de compras?

Con esos datos, podré generar las mejores opciones para ti.

------------------------------------------------------------

Respuesta 2: Perfecto, Franco. Solo me falta saber cuántas personas van a disfrutar de las compras. ¿Cuántas personas incluirás en el carro?


## 6️⃣ Análisis del Comportamiento del Agente

In [19]:
# Verificar historial de memoria
print("📝 HISTORIAL DE CONVERSACIÓN:")
print("="*60)
print(f"Total de mensajes en el historial: {len(chat_history)}")
if chat_history:
    for i, msg in enumerate(chat_history[-6:], 1):  # Últimos 6 mensajes
        tipo = "🧑 Usuario" if isinstance(msg, HumanMessage) else "🤖 Agente"
        print(f"{i}. {tipo}: {msg.content[:100]}...")
        print()
else:
    print("No hay mensajes en el historial aún.")

📝 HISTORIAL DE CONVERSACIÓN:
Total de mensajes en el historial: 0
No hay mensajes en el historial aún.


## 7️⃣ Demostración de Decisiones Adaptativas

### Escenario: Presupuesto Insuficiente

In [20]:
messages = [HumanMessage(content="Necesito un carro fitness para 4 personas pero solo tengo $5,000. ¿Qué me recomiendas?")]
respuesta = agent_executor.invoke({"messages": messages})

print("\n" + "="*60)
print("RESPUESTA ADAPTATIVA DEL AGENTE:")
print("="*60)
print(respuesta['messages'][-1].content)


RESPUESTA ADAPTATIVA DEL AGENTE:
Parece que tu presupuesto es un poco ajustado para un carro de compras fitness para 4 personas. Vamos a intentar optimizarlo.

Primero, te haré algunas preguntas para entender mejor tus necesidades:
1. ¿Quieres incluir algún tipo de restricción dietética (por ejemplo, sin gluten, bajo en carbohidratos)?
2. ¿Qué tipo de productos te gustaría priorizar (proteínas, vegetales, frutas, etc.)?

Con esta información, podré ofrecerte algunas alternativas más económicas. 😊


## 8️⃣ Interfaz de Conversación Interactiva

In [23]:
def chat_con_agente():
    """Interfaz de chat interactivo con el agente."""
    print("\n" + "="*60)
    print("🤖 AGENTE GROZY - Chat Interactivo")
    print("="*60)
    print("Escribe 'salir' para terminar la conversación\n")
    
    conversation_messages = []
    
    while True:
        user_input = input("🧑 Tú: ")
        
        if user_input.lower() in ['salir', 'exit', 'quit']:
            print("\n👋 ¡Hasta luego!")
            break
        
        try:
            # Agregar mensaje del usuario
            conversation_messages.append(HumanMessage(content=user_input))
            
            # Invocar agente
            respuesta = agent_executor.invoke({"messages": conversation_messages})
            
            # Actualizar historial con todos los mensajes
            conversation_messages = respuesta['messages']
            
            # Mostrar respuesta
            print(f"\n🤖 GROZY: {respuesta['messages'][-1].content}\n")
        except Exception as e:
            print(f"\n❌ Error: {str(e)}\n")

# Descomenta la siguiente línea para usar el chat interactivo
# chat_con_agente()

## 9️⃣ Métricas y Evaluación del Agente

In [24]:
print("📊 MÉTRICAS DEL AGENTE GROZY")
print("="*60)
print(f"🔧 Herramientas disponibles: {len(tools)}")
print(f"💾 Productos en base de datos: {len(productos)}")
print(f"📂 Categorías: {len(set(p['categoria'] for p in productos))}")
print(f"🧠 Modelo LLM: gpt-4o-mini")
print(f"🔍 Vector store: FAISS")
print(f"💬 Mensajes en memoria: {len(chat_history)}")
print(f"⚙️ Framework: LangGraph (LangChain 1.0+)")
print(f"\n✅ Agente completamente funcional y operativo")

📊 MÉTRICAS DEL AGENTE GROZY
🔧 Herramientas disponibles: 7
💾 Productos en base de datos: 495
📂 Categorías: 9
🧠 Modelo LLM: gpt-4o-mini
🔍 Vector store: FAISS
💬 Mensajes en memoria: 0
⚙️ Framework: LangGraph (LangChain 1.0+)

✅ Agente completamente funcional y operativo


## ✅ ¡Agente GROZY Funcional!

El agente está completamente operativo usando **LangChain 1.0+ con `create_agent`**.

### 🎯 Características Implementadas:
- ✅ 7 herramientas (consulta, razonamiento, escritura)
- ✅ Planificación adaptativa
- ✅ Memoria conversacional  
- ✅ Optimización de compras
- ✅ Validación dietética y presupuestaria

### 📊 Estadísticas:
- Productos: 495
- Categorías: 9
- Modelo: gpt-4o-mini
- Framework: LangChain 1.0+ / LangGraph